In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time
# from sympy import *
from scipy.linalg import schur, eigvals
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [2]:
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    # ua,svam,vha=la.svd(Am)
    # va=vha.T
    # svam=svam[0]
    # ua,va=np.zeros((N,N)),np.zeros((N,N))
    # ua[:,0]=1.0/np.sqrt(N)
    # va[:NE,0]=JE/np.sqrt(NE)
    # va[NE:,0]=-JI/np.sqrt(NI)
    # va[:,0]/=-np.linalg.norm(va[:,0])
    # svam=np.sqrt(2*(JE**2+JI**2))
    # vha=va.T

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)#,ua,svam,va)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
  Amplit = gavgfix*typenum
  numsample = typenum**2
  Amplitg= np.zeros(numsample)
  idxc=0
  while (1):
    if idxc>=numsample:
      Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
      break
    p=np.random.random(1)
    Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
    if np.sum(Amplitg)>1.0:
      continue 
    elif np.sum(Amplitg)==1.0:
      break
    else:
      idxc +=1
      # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
  # Amplitg=0
  Amplitg*=Amplit
  Amplitg=np.sqrt(Amplitg)
  return Amplitg


In [11]:
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
rho=0.9
JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)#,ua,svam,va
Asym=Am.copy()
Asym[:NE,:NE],Asym[:NE,NE:],Asym[NE:,:NE],Asym[NE:,NE:]=rho*JE/NE,-rho*JI/NE,-rho*JI/NE,-rho*JI/NE
Aasym=Am.copy()-Asym.copy()
eigvalAasym,_=la.eig(Aasym)
eigvalAsym,_=la.eig(Asym)
eigvalA,_=la.eig(Am)
print(Aasym[1::NE,1::NE])
print((1-rho)*JE/NE,(JE+rho*JI)/NE)
print(-(1-rho)*JI/NE,-(1-rho)*JI/NE)

[[ 0.0002 -0.0007]
 [ 0.0038 -0.0007]]
0.00019999999999999996 0.0038000000000000004
-0.0006999999999999999 -0.0006999999999999999


In [17]:
grand = np.zeros(4)
gaverage=0.6
# # max r_g
# grand[0]=gaverage*np.sqrt(2) # E dominant
grand[3]=gaverage*np.sqrt(2) # I dominant
# # totally average
# grand = gaverage*np.ones(4)/np.sqrt(2.0)
# # three copy
# grand = gaverage*np.ones(4)*np.sqrt(2.0/3.0)
# grand[0]=0
X,Xsym,Xasym,Gamp,eigvgmT,eigvecgm,gaverage=generate_localstatsmat_eig_desym(Nparams,grand,rho)
Xsymd,Xasymd,Xd=Xsym.copy(),Xasym.copy(),X.copy()
for i in range(NE*2):
    Xasymd[i,i],Xd[i,i]=0,0
    Xsymd[i,i]=0.0
print(np.sum(np.sum(Xsym.T*Xsym))/2/NE)
print(np.sum(np.sum(Xasym.T*Xasym))/2/NE)
print(np.sum(np.sum(X.T*X))/2/NE)


0.2882300575373627
3.2494717365835934e-05
0.2879484747301734


In [18]:
Xtotal=Am+X.copy()
eigvalsxt,eigvecxt=la.eig(Xtotal)
Xsymtotal=Asym.copy()+Xsym.copy()
eigvalsxst,eigvecxst=la.eig(Xsymtotal)
Xasymtotal=Aasym.copy()+Xasym.copy()
eigvalsxast,eigvecxast=la.eig(Xasymtotal)

fig,ax=plt.subplots(1,3,figsize=(12,3))
ax[0].scatter(np.real(eigvalsxt),np.imag(eigvalsxt),s=5.0,c='blue',alpha=0.5)
ax[0].plot(np.sqrt(eigvgmT[1])*np.ones(20)*(1+rho),np.linspace(-1.0,1.0,20),linewidth=1.5,color='r')
ax[0].plot(np.linspace(-1.0,1.0,20),np.sqrt(eigvgmT[1])*np.ones(20)*(1-rho),linewidth=1.5,color='r')
n, bins, patches = ax[1].hist((eigvalsxst), 100, density=True, alpha=0.5)
# ax[1].plot(2*np.sqrt(rho)*np.ones(20),np.linspace(-1.0,1.0,20),linewidth=1.5,color='r')

# ax[1].hist(np.real(eigvalsxst),np.imag(eigvalsxst))
ax[2].scatter(np.real(eigvalsxast),np.imag(eigvalsxast),s=5.0,c='blue',alpha=0.5)
# ax[2].set_aspect('equal')
# eigvXsym,_=la.eig(Xsym)
# n, bins, patches = ax[3].hist(eigvXsym, 100, density=True, alpha=0.5)


In [22]:
# # svd Xsym
# _,svXsym,_=la.svd(Xsym.copy())
# # eigenspectrum Xsym
# eigvXsym,_=la.eig(Xsym.copy())
# print('compare singular values and eigenvalues:')
# print(svXsym[:2],np.sqrt(svXsym[:2]))
# print(eigvXsym[:2])

# # svd Xasym
# _,svXasym,_=la.svd(Xasym.copy())
# # eigenspectrum Xasym
# eigvXasym,_=la.eig(Xasym.copy())
# print('compare singular values and eigenvalues:')
# print(svXasym[:2],np.sqrt(svXasym[:2]))
# print(eigvXasym[:2])

# # svd Xsym
# _,svXsymtotal,_=la.svd(Xsymtotal.copy())
# print('compare singular values and eigenvalues:')
# print(svXsymtotal[:2])
# print(eigvalsxst[:2])
# # print(np.std(X[:NE,:NE]),np.std(X[:NE,NE:]),np.std(X[NE:,:NE]),np.std(X[NE:,NE:]))

## compare eigenvalues of $\bar{\mathbf{J}}^a$ and eigenvalues of $\mathbf{J}^a$
eigvmeanJa,_=la.eig(Aasym.copy())
print('compare eigenvalues of two asymmetric matrix:')
print(eigvalsxast[:2])
print(eigvmeanJa[:2])

## compare eigenvalues of $\bar{\mathbf{J}}^s$ and eigenvalues of $\mathbf{J}^s$
eigvmeanJs,_=la.eig(Asym.copy())
print('compare eigenvalues of two asymmetric matrix:')
print(eigvalsxst[:2])
print(eigvmeanJs[:2])

compare eigenvalues of two asymmetric matrix:
[-0.2504105+1.56507962j -0.2504105-1.56507962j]
[-0.25+1.56764154j -0.25-1.56764154j]
compare eigenvalues of two asymmetric matrix:
[-4.40684676+0.j  2.40222836+0.j]
[-4.3286634+0.j  2.3286634+0.j]


In [ ]:
fig,ax=plt.subplots(2,1,figsize=(3,12))
centerSym=np.zeros(NE*2)
for i in range(NE*2):
    center
n, bins, patches = ax[0].hist(eigvalAasym, 100, density=True, alpha=0.5)

# generate symmetric random matrix 01/2021

In [35]:
gaverageseries = np.linspace(0.05,1.0,20) # a series of average random gain 
symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
print('g:',gaverageseries[3],gaverageseries[12],gaverageseries[18])
print('eta',symetaseries[2],symetaseries[5],symetaseries[8])

g: 0.2 0.65 0.95
eta 0.2 0.5 0.8


In [35]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])

'''
noise correlation of m and n change with random gain $\bar{g}$ and also
symmetricity $\eta$
'''
# random gain and symmetricity
gaverageseries = np.linspace(0.1,1.0,10) # a series of average random gain 
symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
ngavg,neta=len(gaverageseries),len(symetaseries)
nrank,ntrial=1,20
eigvalseries,mvecseries,nvecseries=np.zeros((ngavg,neta,ntrial,nrank)),np.zeros((ngavg,neta,ntrial,NE*2,nrank)),np.zeros((ngavg,neta,ntrial,NE*2,nrank))
Reigvecseries,Leigvecseries=np.zeros((ngavg,neta,ntrial,nrank)),np.zeros((ngavg,neta,ntrial,nrank))
# generate symmetric random
for igavg, gaverage in enumerate(gaverageseries):
    for jeta,eta in enumerate(symetaseries):
        asqr=(1+np.sqrt(1-eta**2))/2.0
        aamp=np.sqrt(asqr)
        for ktrial in range(ntrial):
            Xinit=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
            X0=Xinit.copy()*gaverage/np.sqrt(2)
            X=aamp*X0.T+np.sqrt(1-aamp**2)*X0
            symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
            # print(symmetricityX)
            # overall
            J = X.copy()+Am.copy()
            # properties
            eigvJ,eigrvecJ=la.eig(J)
            inveigrvecJ=la.inv(eigrvecJ)
            meig = np.squeeze(eigrvecJ[:,:nrank].copy())
            meig = np.reshape(meig,(NE*2,nrank))
            neig = np.squeeze(inveigrvecJ[:nrank,:].copy()) # inverse
            neig = np.reshape(neig.T,(NE*2,nrank))
            # record Reigvec,Leigvec
            Reigvecseries[igavg,jeta,ktrial,:,i]=meig[:,i].copy()
            Leigvecseries[igavg,jeta,ktrial,:,i]=neig[:,i].copy()
            for i in range(nrank):
                neig[:,i]*=eigvJ[i]
                eigvalseries[igavg,jeta,ktrial,i]=eigvJ[i]
                mvecseries[igavg,jeta,ktrial,:,i]=meig[:,i].copy()
                nvecseries[igavg,jeta,ktrial,:,i]=neig[:,i].copy()

            #     print('rank-',i)
            #     print(la.norm(meig[:,i]),la.norm(neig[:,i]))
            # fig,ax=plt.subplots(figsize=(6,6))
            # ax.scatter(meig[:NE],neig[:NE],s=5.0,c='red',alpha=0.25)
            # ax.scatter(meig[NE:],neig[NE:],s=5.0,c='blue',alpha=0.25)
            # ax.set_aspect('equal')

            # recording 
            


KeyboardInterrupt: 

In [6]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE

# gaverageseries = np.linspace(0.05,1.0,20) # a series of average random gain 
# symetaseries = np.linspace(0.0,1.0,3) # a series of symmetricity value
# ngavg,neta=len(gaverageseries),len(symetaseries)
# nrank,ntrial=1,5
# eigvalseries,mvecseries,nvecseries

np.savez(now+'_symmetricity_data',Jparameters=Jparameters,gaverageseries=gaverageseries,symetaseries=symetaseries,mvecseries=mvecseries,nvecseries=nvecseries,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,meigvecAm=meigvecAm,neigvecAm=neigvecAm,eigvAm=eigvAm)

## testing Gaussian distribution; For single gaverage and eta; scatter plot

In [24]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])

'''
noise correlation of m and n change with random gain $\bar{g}$ and also
symmetricity $\eta$
'''
eta = 0.6
asqr=(1+np.sqrt(1-eta**2))/2.0
aamp=np.sqrt(asqr)
gaverage=1.0
Xinit=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
X0=Xinit.copy()*gaverage/np.sqrt(2)
X=aamp*X0.T+np.sqrt(1-aamp**2)*X0
x=np.arange(NE*2)
XX,YY=np.meshgrid(x,x)
idx = np.where(XX<YY)
testvalue = X0[idx]

fig,ax=plt.subplots(figsize=(9,9))
n, bins, patches = ax.hist(np.real(testvalue), 20, density=True, alpha=0.5)


normalization m & n
1.0000000000000004 5.147815070493496
eigenvalues: [-2.5+0.j]


In [28]:
testvalue = X*X.T
sigsym = np.mean(np.reshape(testvalue,-1))
fig,ax=plt.subplots(1,2,figsize=(12,9))
n, bins, patches = ax[0].hist(np.reshape(np.real(testvalue),((NE*2)**2,1)), 100, density=True, alpha=0.5)
testvalue = X0*X0.T
sigAsym = np.mean(np.reshape(testvalue,-1))
# fig,ax=plt.subplots(1,2,figsize=(12,9))
n, bins, patches = ax[1].hist(np.reshape(np.real(testvalue),((NE*2)**2,1)), 100, density=True, alpha=0.5)

## read in data and test how local statistics change with $\bar{g}$ and $\eta$; change with a series --- multiple

In [128]:
strfile ='2021-01-29-05_51_27_symmetricity_data'
data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
mvecseries,nvecseries=data['mvecseries'],data['nvecseries']
gaverageseries=data['gaverageseries']
eigvAm=data['eigvAm']
''' how eigenvalue outlier changes with g_bar '''
# three eta 0.2,0.5,0.8
# np.zeros((ngavg,neta,ntrial,nrank))
ngavg,neta,ntrial,N,nrank=np.shape(mvecseries)
gaverageseries = np.linspace(0.05,1.0,20) # a series of average random gain 
symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
idxg=10
idxeta=np.array([3,6,9])#([2,5,8])
# idxeta1=np.where(symetaseries==0.2)
# idxeta2=np.where(symetaseries==0.5)
# idxeta3=np.where(symetaseries==0.8)
''' get eigenvalue outlier series  '''
eigval_outliers_mean,eigval_outliers_std=np.zeros((neta,ngavg)),np.zeros((neta,ngavg))
eigval_outliers_eta = np.zeros(neta)
idxgavg=10
for i in range(neta):
    eigval_outliers=np.sum(np.squeeze(mvecseries[:,i,:,:,0])*np.squeeze(nvecseries[:,i,:,:,0]),axis=2)
    # print('shape:',np.shape(eigval_outliers))
    eigval_outliers_mean[i,:]=np.mean(eigval_outliers,axis=1)# ngavg,neta,ntrial
    eigval_outliers_eta[i]=np.mean(eigval_outliers_mean[i,:])#eigval_outliers_mean[i,idxgavg]#
    eigval_outliers_std[i,:]=np.std(eigval_outliers,axis=0)
# print('choose eta:',idxeta1,',',idxeta2,',',idxeta3)




In [129]:
fig,ax=plt.subplots(figsize=(6,4))
xticks=np.linspace(0,1.0,3)
xlims=[-0.01,1.06]
yticks=np.linspace(-3.0,-2.5,3)
ylims=[-3.01,-2.49]

# rank-1
clset='rgb'
for i in range(neta):
    ax.plot(gaverageseries,eigval_outliers_mean[i,:], c=(1-symetaseries[i])*np.ones(3))#,label=r'$\eta=$'+str(symetaseries[i]))
ax.plot(gaverageseries,eigvAm[0]*np.ones(ngavg),c='red',linestyle='--',linewidth=2.0)
ax.set_xlim(xlims)
ax.set_xticks(xticks)
ax.set_ylim(ylims)
ax.set_yticks(yticks)
''' how eigenvalue outlier changes with eta '''
fig,ax=plt.subplots(figsize=(6,4))
for i in range(neta):
    ax.scatter(symetaseries[i],eigval_outliers_eta[i], c=(1-symetaseries[i])*np.ones(3),cmap='viridis')
ax.plot(gaverageseries,eigvAm[0]*np.ones(ngavg),c='red',linestyle='--',linewidth=2.0)
ax.set_xlim(xlims)
ax.set_xticks(xticks)
ax.set_ylim(ylims)
ax.set_yticks(yticks)
# ax.set_aspect('equal')

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB o

## noise correlation

In [3]:
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

In [207]:
idxeta,idxtrial=8,8#2#5#8
idxgavg=18#18#12#3#np.array([1,10,19])
meigvec=np.reshape(np.squeeze(mvecseries[idxgavg,idxeta,idxtrial,:,0]),(2*NE,1))
neigvec=np.reshape(np.squeeze(nvecseries[idxgavg,idxeta,idxtrial,:,0]),(2*NE,1))
## validate the normalization
lambdarecon=neigvec.T@meigvec
print('eigenvalue outlier:',lambdarecon)

# fig,ax=plt.subplots(3,3,figsize=(8,8))
# mE,nE=meigvec[:NE,0].copy(),neigvec[:NE,0].copy()
# confidence_ellipse(np.real(nE),np.real(mE),ax[0][0],edgecolor='orange')
# ax[0][0].scatter(nE,mE,s=10,c='red',alpha=0.5)
# mI,nI=meigvec[NE:,0].copy(),neigvec[NE:,0].copy()
# confidence_ellipse(np.real(nI),np.real(mI),ax[0][0],edgecolor='cyan')
# ax[0][0].scatter(nI,mI,s=10,c='blue',alpha=0.5)

eigenvalue outlier: [[-2.82684491]]


In [208]:
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
meigvecloading,neigvecloading=meigvec.copy(),neigvec.copy()
scale_std=3.0
for irank in range(nrank):
    # m vector
    # puring E
    mEmat = np.squeeze(meigvecloading[:NE,irank])
    meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
    varmE = mEmat - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEmat[idxwhere]=meanmEtotal
    meigvecloading[:NE,irank]=mEmat
    meanmE[irank]=np.mean(mEmat)

    # puring I
    mImat = np.squeeze(meigvecloading[NE:,irank])
    meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
    varmI = mImat - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mImat[idxwhere]=meanmItotal
    meigvecloading[NE:,irank]=mImat
    meanmI[irank]=np.mean(mImat)
    
    # n vector
    # puring E
    nEmat = np.squeeze(neigvecloading[:NE,irank])
    meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
    varnE = nEmat - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEmat[idxwhere]=meannEtotal
    neigvecloading[:NE,irank]=nEmat
    meannE[irank]=np.mean(nEmat)

    # puring I
    nImat = np.squeeze(neigvecloading[NE:,irank])
    meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
    varnI = nImat - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nImat[idxwhere]=meannItotal
    neigvecloading[NE:,irank]=nImat
    meannI[irank]=np.mean(nImat)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for i in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)

xlims =[-0.31,0.31]
xticks = np.linspace(-0.3,0.3,3)
ylims =[-0.11,0.11]
yticks = np.linspace(-0.1,0.1,3)
# figure for eigenvectors
fig,ax=plt.subplots(figsize=(12,3))
#
ii,jj=2,2


ax.scatter(neigvecloading[NE:,0],meigvecloading[NE:,0],s=5.0,c='blue',alpha=0.5)
ax.scatter(neigvecloading[:NE,0],meigvecloading[:NE,0],s=5.0,c='red',alpha=0.5)

ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='r',linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='r',linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color='b',linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color='b',linestyle='--',linewidth=1.5)

mE,nE=meigvecloading[:NE,0].copy(),neigvecloading[:NE,0].copy()
confidence_ellipse(np.real(nE),np.real(mE),ax,edgecolor='orange')
ax.scatter(nE,mE,s=10,c='red',alpha=0.5)
mI,nI=meigvecloading[NE:,0].copy(),neigvecloading[NE:,0].copy()
confidence_ellipse(np.real(nI),np.real(mI),ax,edgecolor='cyan')
ax.scatter(nI,mI,s=10,c='blue',alpha=0.5)
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)


ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')

# ax[ii][jj].scatter(neigvecloading[NE:,0],meigvecloading[NE:,0],s=5.0,c='blue',alpha=0.5)
# ax[ii][jj].scatter(neigvecloading[:NE,0],meigvecloading[:NE,0],s=5.0,c='red',alpha=0.5)

# ax[ii][jj].plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='r',linestyle='--',linewidth=1.5)
# ax[ii][jj].plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='r',linestyle='--',linewidth=1.5)
# ax[ii][jj].plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color='b',linestyle='--',linewidth=1.5)
# ax[ii][jj].plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color='b',linestyle='--',linewidth=1.5)

# mE,nE=meigvecloading[:NE,0].copy(),neigvecloading[:NE,0].copy()
# confidence_ellipse(np.real(nE),np.real(mE),ax[ii][jj],edgecolor='orange')
# ax[ii][jj].scatter(nE,mE,s=10,c='red',alpha=0.5)
# mI,nI=meigvecloading[NE:,0].copy(),neigvecloading[NE:,0].copy()
# confidence_ellipse(np.real(nI),np.real(mI),ax[ii][jj],edgecolor='cyan')
# ax[ii][jj].scatter(nI,mI,s=10,c='blue',alpha=0.5)
# ax[ii][jj].axvline(c='grey', lw=1)
# ax[ii][jj].axhline(c='grey', lw=1)


# ax[ii][jj].set_xlim(xlims)
# ax[ii][jj].set_ylim(ylims)
# ax[ii][jj].set_xticks(xticks)
# ax[ii][jj].set_yticks(yticks)
# ax[ii][jj].set_aspect('equal')

(2, 2)
(2, 2)


## heterogeneous connectivity

In [5]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])

'''
noise correlation of m and n change with random gain $\bar{g}$ and also
symmetricity $\eta$
'''
# random gain and symmetricity
gaverageseries = np.linspace(0.1,1.0,10) # a series of average random gain 
symetaseries = np.linspace(0.1,1.0,10) # a series of symmetricity value
ngavg,neta=len(gaverageseries),len(symetaseries)
nrank,ntrial=1,20
eigvalseries,mvecseries,nvecseries=np.zeros((ngavg,neta,ntrial,nrank)),np.zeros((ngavg,neta,ntrial,NE*2,nrank)),np.zeros((ngavg,neta,ntrial,NE*2,nrank))
Reigvecseries,Leigvecseries=np.zeros((ngavg,neta,ntrial,NE*2,nrank)),np.zeros((ngavg,neta,ntrial,NE*2,nrank))
# generate symmetric random
for igavg, gaverage in enumerate(gaverageseries):
    for jeta,eta in enumerate(symetaseries):
        asqr=(1+np.sqrt(1-eta**2))/2.0
        aamp=np.sqrt(asqr)
        for ktrial in range(ntrial):
            Xinit=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
            X0=Xinit.copy()
            X=aamp*X0.T+np.sqrt(1-aamp**2)*X0
            # ''' homogeneous '''
            X*=(gaverage/np.sqrt(2))
            # # ''' heterogeneous rII '''
            # X[:NE,:NE]=0
            # X[:NE,NE:]=0
            # X[NE:,:NE]=0
            # X[NE:,NE:]*=np.sqrt(2)*gaverage
            # # ''' heterogeneous not rEE '''
            # X[:NE,:NE]=0
            # X[:NE,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
            # X[NE:,:NE]*=(np.sqrt(2.0/3.0)*gaverage)
            # X[NE:,NE:]*=(np.sqrt(2.0/3.0)*gaverage)

            symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
            print('g_bar and eta')
            print(gaverage,';',eta)
            print('symmetricity factor')
            print(symmetricityX)
            # overall
            J = X.copy()+Am.copy()
            # properties
            eigvJ,eigrvecJ=la.eig(J)
            inveigrvecJ=la.inv(eigrvecJ)
            meig = np.squeeze(eigrvecJ[:,:nrank].copy())
            meig = np.reshape(meig,(NE*2,nrank))
            neig = np.squeeze(inveigrvecJ[:nrank,:].copy()) # inverse
            neig = np.reshape(neig.T,(NE*2,nrank))
            # record Reigvec,Leigvec
            Reigvecseries[igavg,jeta,ktrial,:,0]=meig[:,0].copy()
            Leigvecseries[igavg,jeta,ktrial,:,0]=neig[:,0].copy()
            for i in range(nrank):
                neig[:,i]*=eigvJ[i]
                eigvalseries[igavg,jeta,ktrial,i]=eigvJ[i]
                mvecseries[igavg,jeta,ktrial,:,i]=meig[:,i].copy()
                nvecseries[igavg,jeta,ktrial,:,i]=neig[:,i].copy()

            # recording 
            


518565641880994
g_bar and eta
0.9 ; 0.5
symmetricity factor
0.40498805089614276
g_bar and eta
0.9 ; 0.5
symmetricity factor
0.40549157502229866
g_bar and eta
0.9 ; 0.6
symmetricity factor
0.4862650342408503
g_bar and eta
0.9 ; 0.6
symmetricity factor
0.4854659534948765
g_bar and eta
0.9 ; 0.6
symmetricity factor
0.48614137619168296
g_bar and eta
0.9 ; 0.6
symmetricity factor
0.4868800345327775
g_bar and eta
0.9 ; 0.6
symmetricity factor
0.4864245010869174
g_bar and eta
0.9 ; 0.6
symmetricity factor
0.48707912229964395
g_bar and eta
0.9 ; 0.6
symmetricity factor
0.48795944284665765
g_bar and eta
0.9 ; 0.6
symmetricity factor
0.48657312566144983
g_bar and eta
0.9 ; 0.6
symmetricity factor
0.4856714820295908
g_bar and eta
0.9 ; 0.6
symmetricity factor
0.48739345268878415
g_bar and eta
0.9 ; 0.6
symmetricity factor
0.4862024365730917
g_bar and eta
0.9 ; 0.6
symmetricity factor
0.48491194591964476
g_bar and eta
0.9 ; 0.6
symmetricity factor
0.48619924283253935
g_bar and eta
0.9 ; 0.6
symmet

In [6]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE

# gaverageseries = np.linspace(0.05,1.0,20) # a series of average random gain 
# symetaseries = np.linspace(0.0,1.0,3) # a series of symmetricity value
# ngavg,neta=len(gaverageseries),len(symetaseries)
# nrank,ntrial=1,5
# eigvalseries,mvecseries,nvecseries

np.savez(now+'_heterNotrEEsymmetricity_data',Jparameters=Jparameters,gaverageseries=gaverageseries,symetaseries=symetaseries,mvecseries=mvecseries,nvecseries=nvecseries,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,meigvecAm=meigvecAm,neigvecAm=neigvecAm,eigvAm=eigvAm)

## 0402 relationship between cov and theta

In [9]:
# strfile ='2021-01-29-05_51_27_symmetricity_data'
# data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
# mvecseries,nvecseries=data['mvecseries'],data['nvecseries']
# gaverageseries=data['gaverageseries']
# eigvAm=data['eigvAm']
''' how eigenvalue outlier changes with g_bar '''
# three eta 0.2,0.5,0.8
# np.zeros((ngavg,neta,ntrial,nrank))
ngavg,neta,ntrial,N,nrank=np.shape(mvecseries)
# gaverageseries = np.linspace(0.05,1.0,20) # a series of average random gain 
# symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
idxg=10
idxeta=np.array([3,6,9])#([2,5,8])
# idxeta1=np.where(symetaseries==0.2)
# idxeta2=np.where(symetaseries==0.5)
# idxeta3=np.where(symetaseries==0.8)
''' get eigenvalue outlier series  '''
eigval_outliers_mean,eigval_outliers_std=np.zeros((neta,ngavg)),np.zeros((neta,ngavg))
eigval_outliers_eta = np.zeros(neta)
idxgavg=10
for i in range(neta):
    # np.zeros((ngavg,neta,ntrial,NE*2,nrank))
    eigval_outliers=np.sum(np.squeeze(mvecseries[:,i,:,:,0])*np.squeeze(nvecseries[:,i,:,:,0]),axis=2)
    # print('shape:',np.shape(eigval_outliers))
    eigval_outliers_mean[i,:]=np.mean(eigval_outliers,axis=1)# ngavg,neta,ntrial
    eigval_outliers_eta[i]=np.mean(eigval_outliers_mean[i,:])#eigval_outliers_mean[i,idxgavg]#
    eigval_outliers_std[i,:]=np.std(eigval_outliers,axis=1)
# print('choose eta:',idxeta1,',',idxeta2,',',idxeta3)




In [10]:
fig,ax=plt.subplots(figsize=(6,4))
xticks=np.linspace(0,1.0,3)
xlims=[-0.01,1.06]
yticks=np.linspace(-3.0,-2.5,3)
ylims=[-3.01,-2.49]

# rank-1
clset='rgb'
for i in range(neta):
    ax.plot(gaverageseries,eigval_outliers_mean[i,:], c=(1-symetaseries[i])*np.ones(3))#,label=r'$\eta=$'+str(symetaseries[i]))
ax.plot(gaverageseries,eigvAm[0]*np.ones(ngavg),c='red',linestyle='--',linewidth=2.0)
ax.set_xlim(xlims)
ax.set_xticks(xticks)
ax.set_ylim(ylims)
ax.set_yticks(yticks)
''' how eigenvalue outlier changes with eta '''
fig,ax=plt.subplots(figsize=(6,4))
for i in range(neta):
    ax.scatter(symetaseries[i],eigval_outliers_eta[i], c=(1-symetaseries[i])*np.ones(3),cmap='viridis')
ax.plot(gaverageseries,eigvAm[0]*np.ones(ngavg),c='red',linestyle='--',linewidth=2.0)
ax.set_xlim(xlims)
ax.set_xticks(xticks)
ax.set_ylim(ylims)
ax.set_yticks(yticks)
# ax.set_aspect('equal')

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB o

In [30]:
thetamaxseries=np.zeros((ngavg,neta,ntrial,2,2))# EI, DIR1,DIR2
print(gaverageseries)
for jdxgavg, gavg in enumerate(gaverageseries):
    for idxeta, eta in enumerate(symetaseries):
        for ktrial in range(ntrial):
            meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
            # print(jdxgavg)
            meigvecloading,neigvecloading=mvecseries[jdxgavg,idxeta,ktrial,:,0].copy(),nvecseries[jdxgavg,idxeta,ktrial,:,0].copy()
            meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
            scale_std=3.0
            for irank in range(nrank):
                # m vector
                # puring E
                mEmat = np.squeeze(meigvecloading[:NE,irank])
                meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
                varmE = mEmat - meanmEtotal
                idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
                mEmat[idxwhere]=meanmEtotal
                meigvecloading[:NE,irank]=mEmat
                meanmE[irank]=np.mean(mEmat)

                # puring I
                mImat = np.squeeze(meigvecloading[NE:,irank])
                meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
                varmI = mImat - meanmItotal
                idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
                mImat[idxwhere]=meanmItotal
                meigvecloading[NE:,irank]=mImat
                meanmI[irank]=np.mean(mImat)
                
                # n vector
                # puring E
                nEmat = np.squeeze(neigvecloading[:NE,irank])
                meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
                varnE = nEmat - meannEtotal
                idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
                nEmat[idxwhere]=meannEtotal
                neigvecloading[:NE,irank]=nEmat
                meannE[irank]=np.mean(nEmat)

                # puring I
                nImat = np.squeeze(neigvecloading[NE:,irank])
                meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
                varnI = nImat - meannItotal
                idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
                nImat[idxwhere]=meannItotal
                neigvecloading[NE:,irank]=nImat
                meannI[irank]=np.mean(nImat)

            ''' calculate directions '''
            noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
            dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
            for i in range(nrank):
                # E 0 M 1 N
                noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
                noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
            # m1n1
            covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
            _,dirvecE=la.eig(covdirE)
            thetamaxseries[jdxgavg,idxeta,ktrial,0,0],thetamaxseries[jdxgavg,idxeta,ktrial,0,1] =np.arctan(dirvecE[0,0]/dirvecE[1,0]),np.arctan(dirvecE[0,1]/dirvecE[1,1])#(dirvecE[0,0]/dirvecE[1,0]),(dirvecE[0,1]/dirvecE[1,1])#
            _,dirvecI=la.eig(covdirI)  
            thetamaxseries[jdxgavg,idxeta,ktrial,1,0],thetamaxseries[jdxgavg,idxeta,ktrial,1,1]  =np.arctan(dirvecI[0,0]/dirvecI[1,0]),np.arctan(dirvecI[0,1]/dirvecI[1,1])#(dirvecI[0,0]/dirvecI[1,0]),(dirvecI[0,1]/dirvecI[1,1])#

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [32]:
fig,ax=plt.subplots(figsize=(4,4))
thetamaxmeanseries=np.mean(thetamaxseries/np.pi*180,axis=2)
# ax.scatter(symetaseries,thetamaxmeanseries[-1,:,0,0],s=5.0,c='blue',alpha=0.5)
ax.scatter(symetaseries,thetamaxmeanseries[-1,:,0,1],s=5.0,c='blue',alpha=0.5)
# ax.plot(symetaseries,eigval_outliers_mean[:,-1],color='cyan')

In [23]:
print(eigval_outliers_eta/thetamaxmeanseries[-1,:,0,0])

[1.78929578 1.6250042  1.64140225 1.65806823 1.67402227 1.70012534
 1.71504522 1.73883777 1.75931238 1.78338465]


In [4]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])

'''
noise correlation of m and n change with random gain $\bar{g}$ and also
symmetricity $\eta$
'''
# random gain and symmetricity
gaverageseries = np.linspace(0.0,1.0,3) # a series of average random gain 
symetaseries = np.linspace(0.0,1.0,3) # a series of symmetricity value
ngavg,neta=len(gaverageseries),len(symetaseries)
nrank,ntrial=1,20
eigvalseries,mvecseries,nvecseries=np.zeros((ntrial,nrank)),np.zeros((ntrial,NE*2,nrank)),np.zeros((ntrial,NE*2,nrank))
Reigvecseries,Leigvecseries=np.zeros((ntrial,NE*2,nrank)),np.zeros((ntrial,NE*2,nrank))
# generate symmetric random
gaverage,eta=gaverageseries[2],symetaseries[2]
# random 
while(1):
    grand = gmatamplitude_eig(gaverage**2,2)
    idxeff = np.where(grand>0)
    if (np.shape(idxeff)[1]>3):
        np.random.shuffle(grand)
        break
print('random amp:',grand)
asqr=(1+np.sqrt(1-eta**2))/2.0
aamp=np.sqrt(asqr)
for ktrial in range(ntrial):
    Xinit=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
    X0=Xinit.copy()#*gaverage#/np.sqrt(2)#1.0/np.sqrt(2)
    X=aamp*X0.T+np.sqrt(1-aamp**2)*X0
    # # ''' homogeneous '''
    # X*=(gaverage/np.sqrt(2))
    # # ''' heterogeneous rII '''
    # X[:NE,:NE]=0
    # X[:NE,NE:]=0
    # X[NE:,:NE]=0
    # X[NE:,NE:]*=np.sqrt(2)*gaverage
    # # ''' heterogeneous not rEE '''
    # X[:NE,:NE]=0
    # X[:NE,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
    # X[NE:,:NE]*=(np.sqrt(2.0/3.0)*gaverage)
    # X[NE:,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
    # random 
    X[:NE,:NE]*=grand[0]
    X[:NE,NE:]*=grand[1]
    X[NE:,:NE]*=grand[2]
    X[NE:,NE:]*=grand[3]
    theogrand = (grand[1]*grand[2]*2+grand[0]**2+grand[3]**2)/2
    symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
    print(symmetricityX,theogrand*eta)
    # overall
    J = X.copy()+Am.copy()
    # properties
    eigvJ,eigrvecJ=la.eig(J)
    inveigrvecJ=la.inv(eigrvecJ)
    meig = np.squeeze(eigrvecJ[:,:nrank].copy())
    meig = np.reshape(meig,(NE*2,nrank))
    neig = np.squeeze(inveigrvecJ[:nrank,:].copy()) # inverse
    neig = np.reshape(neig.T,(NE*2,nrank))
    # record Reigvec,Leigvec
    Reigvecseries[ktrial,:,0]=meig[:,0].copy()
    Leigvecseries[ktrial,:,0]=neig[:,0].copy()
    for i in range(nrank):
        neig[:,i]*=eigvJ[i]
        eigvalseries[ktrial,i]=eigvJ[i]
        mvecseries[ktrial,:,i]=meig[:,i].copy()
        nvecseries[ktrial,:,i]=neig[:,i].copy()

            


normalization m & n
1.0000000000000004 5.147815070493496
eigenvalues: [-2.5+0.j]
random amp: [0.72253801 0.37721751 0.2633523  1.12529611]
0.9945709711342083 0.9935173578199656
0.993788962836967 0.9935173578199656
0.9948637211099867 0.9935173578199656
0.9926942803320052 0.9935173578199656
0.992495917805108 0.9935173578199656
0.9948098351076644 0.9935173578199656
0.9957230046929537 0.9935173578199656
0.9942258915277955 0.9935173578199656
0.9962091123023292 0.9935173578199656
0.993624806713646 0.9935173578199656
0.9937525632295863 0.9935173578199656
0.9951125341478939 0.9935173578199656
0.9945431094953139 0.9935173578199656
0.9939720399732624 0.9935173578199656
0.9933457661227937 0.9935173578199656
0.9965029633512769 0.9935173578199656
0.9945177642032687 0.9935173578199656
0.9928388673018617 0.9935173578199656
0.9943149642704904 0.9935173578199656
0.9973419171407758 0.9935173578199656


In [39]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE

gparameters=np.zeros(4+2)
gparameters[:4],gparameters[4],gparameters[5]=grand,gaverage,eta
np.savez(now+'_hetersymmetricity_data',Jparameters=Jparameters,gparameters=gparameters,mvecseries=mvecseries,nvecseries=nvecseries,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,meigvecAm=meigvecAm,neigvecAm=neigvecAm,eigvAm=eigvAm)

In [8]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE

# gaverageseries = np.linspace(0.05,1.0,20) # a series of average random gain 
# symetaseries = np.linspace(0.0,1.0,3) # a series of symmetricity value
# ngavg,neta=len(gaverageseries),len(symetaseries)
# nrank,ntrial=1,5
# eigvalseries,mvecseries,nvecseries

np.savez(now+'_hetersymmetricity_data',Jparameters=Jparameters,gaverageseries=gaverageseries,symetaseries=symetaseries,mvecseries=mvecseries,nvecseries=nvecseries,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,meigvecAm=meigvecAm,neigvecAm=neigvecAm,eigvAm=eigvAm)

[0.  0.5 1. ] [0.05 1.  ]


## RANDOM $r_g$, figuring out the relationship between overlap and eigenvalue outlier in the general case

In [47]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])

'''
fixed g_bar but r_g changes (randomly insert)
eta changes and find out the relationships with eta, rotation and eigenvalue outliers
* symetaseries 
* gaverages =1.0
* grand fixed 
* ntrials
'''
# random gain and symmetricity
gaverage=1.0
symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
nrank,ntrial,neta=1,20,len(symetaseries)

# original data
eigvalseries,mvecseries,nvecseries=np.zeros((neta,ntrial,nrank)),np.zeros((neta,ntrial,NE*2,nrank)),np.zeros((neta,ntrial,NE*2,nrank))
Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,nrank)),np.zeros((neta,ntrial,NE*2,nrank))
# properties
thetamaxseries=np.zeros((neta,ntrial,2,2))
# random 
while(1):
    grand = gmatamplitude_eig(gaverage**2,2)
    idxeff = np.where(grand>0)
    if (np.shape(idxeff)[1]>3):
        np.random.shuffle(grand)
        break
grandmax,grandmin=np.max(grand),-np.max(-grand)
grandother=np.setdiff1d(grand,grandmax)
grandother=np.setdiff1d(grandother,grandmin)
grand[0],grand[3],grand[1],grand[2]=grandmax,grandmin,grandother[0],grandother[1]
print('random amp:',grand)
''' test r_g and gbar'''
# theoretical M (grandom) matrix 
xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
# first do not multiply at
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print('gbar:,',gaverage_,'; effective gain:',r_g)
for idxeta, eta in enumerate(symetaseries):
    asqr=(1+np.sqrt(1-eta**2))/2.0
    aamp=np.sqrt(asqr)
    for ktrial in range(ntrial):
        Xinit=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
        X0=Xinit.copy()#*gaverage#/np.sqrt(2)#1.0/np.sqrt(2)
        X=aamp*X0.T+np.sqrt(1-aamp**2)*X0
        # # ''' homogeneous '''
        # X*=(gaverage/np.sqrt(2))
        # # ''' heterogeneous rII '''
        # X[:NE,:NE]=0
        # X[:NE,NE:]=0
        # X[NE:,:NE]=0
        # X[NE:,NE:]*=np.sqrt(2)*gaverage
        # # ''' heterogeneous not rEE '''
        # X[:NE,:NE]=0
        # X[:NE,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
        # X[NE:,:NE]*=(np.sqrt(2.0/3.0)*gaverage)
        # X[NE:,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
        # random 
        X[:NE,:NE]*=grand[0]
        X[:NE,NE:]*=grand[1]
        X[NE:,:NE]*=grand[2]
        X[NE:,NE:]*=grand[3]
        theogrand = (grand[1]*grand[2]*2+grand[0]**2+grand[3]**2)/2
        symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
        print(symmetricityX,theogrand*eta)
        # overall
        J = X.copy()+Am.copy()
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:nrank].copy())
        meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:nrank,:].copy()) # inverse
        neig = np.reshape(neig.T,(NE*2,nrank))
        # record Reigvec,Leigvec
        Reigvecseries[idxeta,ktrial,:,0]=meig[:,0].copy()
        Leigvecseries[idxeta,ktrial,:,0]=neig[:,0].copy()
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
            eigvalseries[idxeta,ktrial,i]=eigvJ[i]
            mvecseries[idxeta,ktrial,:,i]=meig[:,i].copy()
            nvecseries[idxeta,ktrial,:,i]=neig[:,i].copy()
        meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
        meigvecloading,neigvecloading=mvecseries[idxeta,ktrial,:,0].copy(),nvecseries[idxeta,ktrial,:,0].copy()
        meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
        scale_std=3.0
        for irank in range(nrank):
            # m vector
            # puring E
            mEmat = np.squeeze(meigvecloading[:NE,irank])
            meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
            varmE = mEmat - meanmEtotal
            idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
            mEmat[idxwhere]=meanmEtotal
            meigvecloading[:NE,irank]=mEmat
            meanmE[irank]=np.mean(mEmat)

            # puring I
            mImat = np.squeeze(meigvecloading[NE:,irank])
            meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
            varmI = mImat - meanmItotal
            idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
            mImat[idxwhere]=meanmItotal
            meigvecloading[NE:,irank]=mImat
            meanmI[irank]=np.mean(mImat)
            
            # n vector
            # puring E
            nEmat = np.squeeze(neigvecloading[:NE,irank])
            meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
            varnE = nEmat - meannEtotal
            idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
            nEmat[idxwhere]=meannEtotal
            neigvecloading[:NE,irank]=nEmat
            meannE[irank]=np.mean(nEmat)

            # puring I
            nImat = np.squeeze(neigvecloading[NE:,irank])
            meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
            varnI = nImat - meannItotal
            idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
            nImat[idxwhere]=meannItotal
            neigvecloading[NE:,irank]=nImat
            meannI[irank]=np.mean(nImat)

        ''' calculate directions '''
        noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
        dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
        for i in range(nrank):
            # E 0 M 1 N
            noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
            noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
        # m1n1
        covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
        _,dirvecE=la.eig(covdirE)
        thetamaxseries[idxeta,ktrial,0,0],thetamaxseries[idxeta,ktrial,0,1]  =np.arctan2(dirvecE[1,0],dirvecE[0,0]),np.arctan2(dirvecE[1,1],dirvecE[0,1])
        _,dirvecI=la.eig(covdirI)  
        thetamaxseries[idxeta,ktrial,1,0],thetamaxseries[idxeta,ktrial,1,1]  =np.arctan2(dirvecI[1,0],dirvecI[0,0]),np.arctan2(dirvecI[1,1],dirvecI[0,1])   
            


normalization m & n
1.0000000000000004 5.147815070493496
eigenvalues: [-2.5+0.j]
random amp: [1.08058364 0.42681251 0.80121589 0.09068169]
gbar:, 0.9999999999999999 ; effective gain: 1.122945821563357
0.0006073115922544233 0.0
-0.0013401019894985452 0.0
-0.0002912066884999326 0.0
-0.00010305199737813161 0.0
-0.0001152396480958602 0.0
-5.3054495180558196e-05 0.0
0.00047593582922458594 0.0
0.0008090445273724967 0.0
-0.0005653939961080425 0.0
0.0005878023350377814 0.0
0.0010087364204772362 0.0
0.00011909649451880462 0.0
-0.00010456809525493578 0.0
0.0013652518553847433 0.0
0.0003743518969908566 0.0
0.0010074836421207337 0.0
4.4561404542605736e-05 0.0
-0.0006537552127324736 0.0
-0.00034833976445331426 0.0
0.0007420325486594375 0.0
0.09336731132136905 0.09299110549552853
0.09128560433775017 0.09299110549552853
0.09369783327693873 0.09299110549552853
0.09238188259804042 0.09299110549552853
0.09331765509924923 0.09299110549552853
0.09307637208806557 0.09299110549552853
0.09471959995798711 0.0

In [48]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE

gparameters=np.zeros(4+2+1)
gparameters[:4],gparameters[4],gparameters[5],gparameters[6]=grand,gaverage,r_g,eta
np.savez(now+'_hetersymmetricity_data',Jparameters=Jparameters,gparameters=gparameters,mvecseries=mvecseries,nvecseries=nvecseries,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,meigvecAm=meigvecAm,neigvecAm=neigvecAm,eigvAm=eigvAm,thetamaxseries=thetamaxseries)

In [49]:
for idxeta, eta in enumerate(symetaseries): 
    ktrial=6

    meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
    meigvecloading,neigvecloading=mvecseries[idxeta,ktrial,:,0].copy(),nvecseries[idxeta,ktrial,:,0].copy()
    meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
    scale_std=3.0
    for irank in range(nrank):
        # m vector
        # puring E
        mEmat = np.squeeze(meigvecloading[:NE,irank])
        meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
        varmE = mEmat - meanmEtotal
        idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
        mEmat[idxwhere]=meanmEtotal
        meigvecloading[:NE,irank]=mEmat
        meanmE[irank]=np.mean(mEmat)

        # puring I
        mImat = np.squeeze(meigvecloading[NE:,irank])
        meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
        varmI = mImat - meanmItotal
        idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
        mImat[idxwhere]=meanmItotal
        meigvecloading[NE:,irank]=mImat
        meanmI[irank]=np.mean(mImat)
        
        # n vector
        # puring E
        nEmat = np.squeeze(neigvecloading[:NE,irank])
        meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
        varnE = nEmat - meannEtotal
        idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
        nEmat[idxwhere]=meannEtotal
        neigvecloading[:NE,irank]=nEmat
        meannE[irank]=np.mean(nEmat)

        # puring I
        nImat = np.squeeze(neigvecloading[NE:,irank])
        meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
        varnI = nImat - meannItotal
        idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
        nImat[idxwhere]=meannItotal
        neigvecloading[NE:,irank]=nImat
        meannI[irank]=np.mean(nImat)

    ''' calculate directions '''
    noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
    dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
    for i in range(nrank):
        # E 0 M 1 N
        noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
        noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
    # m1n1
    covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
    _,dirvecE=la.eig(covdirE)
    _,dirvecI=la.eig(covdirI)  
    
    fig,ax=plt.subplots(figsize=(12,3))  
    ax.scatter(neigvecloading[NE:,0],meigvecloading[NE:,0],s=5.0,c='blue',alpha=0.5)
    ax.scatter(neigvecloading[:NE,0],meigvecloading[:NE,0],s=5.0,c='red',alpha=0.5)

    ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='r',linestyle='--',linewidth=1.5)
    ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='r',linestyle='--',linewidth=1.5)
    ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color='b',linestyle='--',linewidth=1.5)
    ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color='b',linestyle='--',linewidth=1.5)

    mE,nE=meigvecloading[:NE,0].copy(),neigvecloading[:NE,0].copy()
    confidence_ellipse(np.real(nE),np.real(mE),ax,edgecolor='orange')
    ax.scatter(nE,mE,s=10,c='red',alpha=0.5)
    mI,nI=meigvecloading[NE:,0].copy(),neigvecloading[NE:,0].copy()
    confidence_ellipse(np.real(nI),np.real(mI),ax,edgecolor='cyan')
    ax.scatter(nI,mI,s=10,c='blue',alpha=0.5)
    ax.axvline(c='grey', lw=1)
    ax.axhline(c='grey', lw=1)
    xxx=np.linspace(0.0,1.0,10)
    yyy=np.tan(thetamaxseries[idxeta,6,0,0])*xxx
    ax.plot(xxx,yyy,color='green',linestyle='--',linewidth=2.0)
    # ax.set_xlim(xlims)
    # ax.set_ylim(ylims)
    # ax.set_xticks(xticks)
    # ax.set_yticks(yticks)
    ax.set_aspect('equal')
            


(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)


In [50]:
fig,ax=plt.subplots(figsize=(12,3))  
ax.plot(np.mean(np.squeeze(eigvalseries[:,:,0]),axis=1),np.mean(np.squeeze(thetamaxseries[:,:,0,0]),axis=1),color='red',alpha=0.5)
ax.plot(np.mean(np.squeeze(eigvalseries[:,:,0]),axis=1),np.mean(np.squeeze(thetamaxseries[:,:,1,0]),axis=1),color='blue',alpha=0.5)
print(grand)

[1.08058364 0.42681251 0.80121589 0.09068169]


## More generalized case with $-1\leq\eta<0$ (C0 rules in paper)

In [36]:
eta = 0.6
Xsym=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
XsymT=Xsym.copy().T
Yidendown=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
Yidenup=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
X=np.arange(NE*2)
XX,YY=np.meshgrid(X,X)
idxdowntriangle=np.where(XX>YY)
idxuptriangle=np.where(XX<YY)
Xinit = Xsym.copy()
Xinit[idxdowntriangle]=Xsym[idxdowntriangle]*np.sqrt(eta)+Yidendown[idxdowntriangle].copy()*np.sqrt(1-eta)
Xinit[idxuptriangle]=-XsymT[idxuptriangle]*np.sqrt(eta)+Yidenup[idxuptriangle].copy()*np.sqrt(1-eta)
Xinit=Xinit.copy()/np.sqrt(2)

eta_reconstruct = np.sum(np.sum(Xinit*Xinit.T))/(NE*NE*4)*NE*2
print('compare eta:',eta,'; reconstruct:',eta_reconstruct)


compare eta: 0.6 ; reconstruct: -0.5985395373543778


In [46]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])

'''
fixed g_bar but r_g changes (randomly insert)
eta changes and find out the relationships with eta, rotation and eigenvalue outliers
* symetaseries 
* gaverages =1.0
* grand fixed 
* ntrials
'''
# random gain and symmetricity
gaverage=1.0
symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
nrank,ntrial,neta=1,20,len(symetaseries)
''' C0 rules '''
xax=np.arange(NE*2)
XX,YY=np.meshgrid(xax,xax)
idxdowntriangle=np.where(XX>YY)
idxuptriangle=np.where(XX<YY)
# original data
eigvalseries,mvecseries,nvecseries=np.zeros((neta,ntrial,nrank)),np.zeros((neta,ntrial,NE*2,nrank)),np.zeros((neta,ntrial,NE*2,nrank))
Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,nrank)),np.zeros((neta,ntrial,NE*2,nrank))
# properties
thetamaxseries=np.zeros((neta,ntrial,2,2))
# # random 
# while(1):
#     grand = gmatamplitude_eig(gaverage**2,2)
#     idxeff = np.where(grand>0)
#     if (np.shape(idxeff)[1]>3):
#         np.random.shuffle(grand)
#         break
# print('random amp:',grand)
# ''' test r_g and gbar'''
# # theoretical M (grandom) matrix 
# xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
# gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
# # first do not multiply at
# gaverage_=0
# for i in range(2):
#     for j in range(2):
#         gaverage_+=gmat[i,j]/2 # ntype=2
# gaverage_=np.sqrt(gaverage_)
# eigvgm,eigvecgm=la.eig(gmat) 
# r_g2=np.max(eigvgm)
# r_g = np.sqrt(r_g2)
# print('gbar:,',gaverage_,'; effective gain:',r_g)
for idxeta, eta in enumerate(symetaseries):
    for ktrial in range(ntrial):
        Xsym=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
        XsymT=Xsym.copy().T
        Yidendown=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
        Yidenup=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
        Xinit = Xsym.copy()
        Xinit[idxdowntriangle]=Xsym[idxdowntriangle]*np.sqrt(eta)+Yidendown[idxdowntriangle].copy()*np.sqrt(1-eta)
        Xinit[idxuptriangle]=-XsymT[idxuptriangle]*np.sqrt(eta)+Yidenup[idxuptriangle].copy()*np.sqrt(1-eta)
        # Xinit=Xinit.copy()/np.sqrt(2)
        X=Xinit.copy()
        # # ''' homogeneous '''
        # X*=(gaverage/np.sqrt(2))
        # # ''' heterogeneous rII '''
        # X[:NE,:NE]=0
        # X[:NE,NE:]=0
        # X[NE:,:NE]=0
        # X[NE:,NE:]*=np.sqrt(2)*gaverage
        # ''' heterogeneous not rEE '''
        X[:NE,:NE]=0
        X[:NE,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
        X[NE:,:NE]*=(np.sqrt(2.0/3.0)*gaverage)
        X[NE:,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
        # # random 
        # X[:NE,:NE]*=grand[0]
        # X[:NE,NE:]*=grand[1]
        # X[NE:,:NE]*=grand[2]
        # X[NE:,NE:]*=grand[3]
        # theogrand = (grand[1]*grand[2]*2+grand[0]**2+grand[3]**2)/2
        symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
        print(symmetricityX,eta)
        # overall
        J = X.copy()+Am.copy()
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:nrank].copy())
        meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:nrank,:].copy()) # inverse
        neig = np.reshape(neig.T,(NE*2,nrank))
        # record Reigvec,Leigvec
        Reigvecseries[idxeta,ktrial,:,0]=meig[:,0].copy()
        Leigvecseries[idxeta,ktrial,:,0]=neig[:,0].copy()
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
            eigvalseries[idxeta,ktrial,i]=eigvJ[i]
            mvecseries[idxeta,ktrial,:,i]=meig[:,i].copy()
            nvecseries[idxeta,ktrial,:,i]=neig[:,i].copy()
        meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
        meigvecloading,neigvecloading=mvecseries[idxeta,ktrial,:,0].copy(),nvecseries[idxeta,ktrial,:,0].copy()
        meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
        scale_std=3.0
        for irank in range(nrank):
            # m vector
            # puring E
            mEmat = np.squeeze(meigvecloading[:NE,irank])
            meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
            varmE = mEmat - meanmEtotal
            idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
            mEmat[idxwhere]=meanmEtotal
            meigvecloading[:NE,irank]=mEmat
            meanmE[irank]=np.mean(mEmat)

            # puring I
            mImat = np.squeeze(meigvecloading[NE:,irank])
            meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
            varmI = mImat - meanmItotal
            idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
            mImat[idxwhere]=meanmItotal
            meigvecloading[NE:,irank]=mImat
            meanmI[irank]=np.mean(mImat)
            
            # n vector
            # puring E
            nEmat = np.squeeze(neigvecloading[:NE,irank])
            meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
            varnE = nEmat - meannEtotal
            idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
            nEmat[idxwhere]=meannEtotal
            neigvecloading[:NE,irank]=nEmat
            meannE[irank]=np.mean(nEmat)

            # puring I
            nImat = np.squeeze(neigvecloading[NE:,irank])
            meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
            varnI = nImat - meannItotal
            idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
            nImat[idxwhere]=meannItotal
            neigvecloading[NE:,irank]=nImat
            meannI[irank]=np.mean(nImat)

        ''' calculate directions '''
        noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
        dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
        for i in range(nrank):
            # E 0 M 1 N
            noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
            noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
        # m1n1
        covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
        _,dirvecE=la.eig(covdirE)
        thetamaxseries[idxeta,ktrial,0,0],thetamaxseries[idxeta,ktrial,0,1]  =np.arctan2(dirvecE[1,0],dirvecE[0,0]),np.arctan2(dirvecE[1,1],dirvecE[0,1])
        _,dirvecI=la.eig(covdirI)  
        thetamaxseries[idxeta,ktrial,1,0],thetamaxseries[idxeta,ktrial,1,1]  =np.arctan2(dirvecI[1,0],dirvecI[0,0]),np.arctan2(dirvecI[1,1],dirvecI[0,1])   
            


normalization m & n
1.0000000000000004 5.147815070493496
eigenvalues: [-2.5+0.j]
0.0008055097215900666 0.0
0.001740523124995603 0.0
0.001563117063333889 0.0
0.0006378576169425221 0.0
0.00038557931694830564 0.0
-0.0008050109895984351 0.0
-0.00018017258477384654 0.0
0.000400055072098016 0.0
0.0012850504830523334 0.0
-0.0002761346872093815 0.0
0.0006713752183875737 0.0
0.0008350161941570148 0.0
0.0002775381897842253 0.0
-4.656241387427208e-05 0.0
0.00041320931790012804 0.0
-0.0004260918963999249 0.0
0.0005796133126893948 0.0
0.0006468126488328053 0.0
-0.00035353729564631143 0.0
2.052269130274115e-05 0.0
-0.09933961475146316 0.1
-0.09975327141212911 0.1
-0.0976138731896334 0.1
-0.10004261779138589 0.1
-0.09798691570394838 0.1
-0.10117612491413328 0.1
-0.10042495971001966 0.1
-0.09940298072826616 0.1
-0.09959104869581696 0.1
-0.10013341403749183 0.1
-0.09977085384561726 0.1
-0.09991655491053868 0.1
-0.09906859340086545 0.1
-0.10086396148569976 0.1
-0.09910811145778184 0.1
-0.100237081631377

In [44]:
for idxeta, eta in enumerate(symetaseries): 
    ktrial=6

    meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
    meigvecloading,neigvecloading=mvecseries[idxeta,ktrial,:,0].copy(),nvecseries[idxeta,ktrial,:,0].copy()
    meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
    scale_std=3.0
    for irank in range(nrank):
        # m vector
        # puring E
        mEmat = np.squeeze(meigvecloading[:NE,irank])
        meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
        varmE = mEmat - meanmEtotal
        idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
        mEmat[idxwhere]=meanmEtotal
        meigvecloading[:NE,irank]=mEmat
        meanmE[irank]=np.mean(mEmat)

        # puring I
        mImat = np.squeeze(meigvecloading[NE:,irank])
        meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
        varmI = mImat - meanmItotal
        idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
        mImat[idxwhere]=meanmItotal
        meigvecloading[NE:,irank]=mImat
        meanmI[irank]=np.mean(mImat)
        
        # n vector
        # puring E
        nEmat = np.squeeze(neigvecloading[:NE,irank])
        meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
        varnE = nEmat - meannEtotal
        idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
        nEmat[idxwhere]=meannEtotal
        neigvecloading[:NE,irank]=nEmat
        meannE[irank]=np.mean(nEmat)

        # puring I
        nImat = np.squeeze(neigvecloading[NE:,irank])
        meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
        varnI = nImat - meannItotal
        idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
        nImat[idxwhere]=meannItotal
        neigvecloading[NE:,irank]=nImat
        meannI[irank]=np.mean(nImat)

    ''' calculate directions '''
    noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
    dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
    for i in range(nrank):
        # E 0 M 1 N
        noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
        noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
    # m1n1
    covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
    _,dirvecE=la.eig(covdirE)
    _,dirvecI=la.eig(covdirI)  
    
    fig,ax=plt.subplots(figsize=(12,3))  
    ax.scatter(neigvecloading[NE:,0],meigvecloading[NE:,0],s=5.0,c='blue',alpha=0.5)
    ax.scatter(neigvecloading[:NE,0],meigvecloading[:NE,0],s=5.0,c='red',alpha=0.5)

    ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='r',linestyle='--',linewidth=1.5)
    ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='r',linestyle='--',linewidth=1.5)
    ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color='b',linestyle='--',linewidth=1.5)
    ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color='b',linestyle='--',linewidth=1.5)

    mE,nE=meigvecloading[:NE,0].copy(),neigvecloading[:NE,0].copy()
    confidence_ellipse(np.real(nE),np.real(mE),ax,edgecolor='orange')
    ax.scatter(nE,mE,s=10,c='red',alpha=0.5)
    mI,nI=meigvecloading[NE:,0].copy(),neigvecloading[NE:,0].copy()
    confidence_ellipse(np.real(nI),np.real(mI),ax,edgecolor='cyan')
    ax.scatter(nI,mI,s=10,c='blue',alpha=0.5)
    ax.axvline(c='grey', lw=1)
    ax.axhline(c='grey', lw=1)
    xxx=np.linspace(0.0,1.0,10)
    yyy=np.tan(thetamaxseries[idxeta,6,0,0])*xxx
    ax.plot(xxx,yyy,color='green',linestyle='--',linewidth=2.0)
    # ax.set_xlim(xlims)
    # ax.set_ylim(ylims)
    # ax.set_xticks(xticks)
    # ax.set_yticks(yticks)
    ax.set_aspect('equal')
            


(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)


In [45]:
fig,ax=plt.subplots(figsize=(12,3))  
ax.plot(np.mean(np.squeeze(eigvalseries[:,:,0]),axis=1),np.mean(np.squeeze(thetamaxseries[:,:,0,0]),axis=1),color='red',alpha=0.5)
ax.plot(np.mean(np.squeeze(eigvalseries[:,:,0]),axis=1),np.mean(np.squeeze(thetamaxseries[:,:,1,0]),axis=1),color='blue',alpha=0.5)
print(grand)

[0.2217127  1.27866169 0.25932493 0.49861642]


In [42]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE

gparameters=np.zeros(4+2+1)
gparameters[:4],gparameters[4],gparameters[5],gparameters[6]=grand,gaverage,r_g,eta
np.savez(now+'_hetersymmetricityNeg_data',Jparameters=Jparameters,gparameters=gparameters,mvecseries=mvecseries,nvecseries=nvecseries,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,meigvecAm=meigvecAm,neigvecAm=neigvecAm,eigvAm=eigvAm,thetamaxseries=thetamaxseries)

## heterogeneous degrees of symmetry

In [52]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])

'''
fixed g_bar but r_g changes (randomly insert)
eta changes and find out the relationships with eta, rotation and eigenvalue outliers
* symetaseries 
* gaverages =1.0
* grand fixed 
* ntrials
'''
''' ---------- method 2 ------------------ '''
# random gain and symmetricity
gaverage=1.0
symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
nrank,ntrial,neta=1,2,len(symetaseries)
''' C0 rules '''
xax=np.arange(NE*2)
XX,YY=np.meshgrid(xax,xax)
idxdowntriangle=np.where(XX>YY)
idxuptriangle=np.where(XX<YY)
# original data
eigvalseries,mvecseries,nvecseries=np.zeros((neta,ntrial,nrank)),np.zeros((neta,ntrial,NE*2,nrank)),np.zeros((neta,ntrial,NE*2,nrank))
Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,nrank)),np.zeros((neta,ntrial,NE*2,nrank))
# properties
thetamaxseries=np.zeros((neta,ntrial,2,2))
# # random 
# while(1):
#     grand = gmatamplitude_eig(gaverage**2,2)
#     idxeff = np.where(grand>0)
#     if (np.shape(idxeff)[1]>3):
#         np.random.shuffle(grand)
#         break
# print('random amp:',grand)
# ''' test r_g and gbar'''
# # theoretical M (grandom) matrix 
# xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
# gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
# # first do not multiply at
# gaverage_=0
# for i in range(2):
#     for j in range(2):
#         gaverage_+=gmat[i,j]/2 # ntype=2
# gaverage_=np.sqrt(gaverage_)
# eigvgm,eigvecgm=la.eig(gmat) 
# r_g2=np.max(eigvgm)
# r_g = np.sqrt(r_g2)
# print('gbar:,',gaverage_,'; effective gain:',r_g)
for idxeta, eta in enumerate(symetaseries):
    for ktrial in range(ntrial):
        Xsym=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
        XsymT=Xsym.copy().T
        Yidendown=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
        Yidenup=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
        Xinit = Xsym.copy()
        Xinit[idxdowntriangle]=Xsym[idxdowntriangle]*np.sqrt(eta)+Yidendown[idxdowntriangle].copy()*np.sqrt(1-eta)
        Xinit[idxuptriangle]=XsymT[idxuptriangle]*np.sqrt(eta)+Yidenup[idxuptriangle].copy()*np.sqrt(1-eta)
        Yind=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
        # Xinit=Xinit.copy()/np.sqrt(2)
        X=Yind.copy()
        X[:NE,:NE]=Xinit[:NE,:NE].copy()
        # ''' homogeneous '''
        X*=(gaverage/np.sqrt(2))
        # # ''' heterogeneous rII '''
        # X[:NE,:NE]=0
        # X[:NE,NE:]=0
        # X[NE:,:NE]=0
        # X[NE:,NE:]*=np.sqrt(2)*gaverage
        # # ''' heterogeneous not rEE '''
        # X[:NE,:NE]=0
        # X[:NE,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
        # X[NE:,:NE]*=(np.sqrt(2.0/3.0)*gaverage)
        # X[NE:,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
        # # random 
        # X[:NE,:NE]*=grand[0]
        # X[:NE,NE:]*=grand[1]
        # X[NE:,:NE]*=grand[2]
        # X[NE:,NE:]*=grand[3]
        # theogrand = (grand[1]*grand[2]*2+grand[0]**2+grand[3]**2)/2
        symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
        print(symmetricityX,eta)
        # overall
        J = X.copy()+Am.copy()
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:nrank].copy())
        meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:nrank,:].copy()) # inverse
        neig = np.reshape(neig.T,(NE*2,nrank))
        # record Reigvec,Leigvec
        Reigvecseries[idxeta,ktrial,:,0]=meig[:,0].copy()
        Leigvecseries[idxeta,ktrial,:,0]=neig[:,0].copy()
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
            eigvalseries[idxeta,ktrial,i]=eigvJ[i]
            mvecseries[idxeta,ktrial,:,i]=meig[:,i].copy()
            nvecseries[idxeta,ktrial,:,i]=neig[:,i].copy()
        meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
        meigvecloading,neigvecloading=mvecseries[idxeta,ktrial,:,0].copy(),nvecseries[idxeta,ktrial,:,0].copy()
        meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
        scale_std=3.0
        for irank in range(nrank):
            # m vector
            # puring E
            mEmat = np.squeeze(meigvecloading[:NE,irank])
            meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
            varmE = mEmat - meanmEtotal
            idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
            mEmat[idxwhere]=meanmEtotal
            meigvecloading[:NE,irank]=mEmat
            meanmE[irank]=np.mean(mEmat)

            # puring I
            mImat = np.squeeze(meigvecloading[NE:,irank])
            meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
            varmI = mImat - meanmItotal
            idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
            mImat[idxwhere]=meanmItotal
            meigvecloading[NE:,irank]=mImat
            meanmI[irank]=np.mean(mImat)
            
            # n vector
            # puring E
            nEmat = np.squeeze(neigvecloading[:NE,irank])
            meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
            varnE = nEmat - meannEtotal
            idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
            nEmat[idxwhere]=meannEtotal
            neigvecloading[:NE,irank]=nEmat
            meannE[irank]=np.mean(nEmat)

            # puring I
            nImat = np.squeeze(neigvecloading[NE:,irank])
            meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
            varnI = nImat - meannItotal
            idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
            nImat[idxwhere]=meannItotal
            neigvecloading[NE:,irank]=nImat
            meannI[irank]=np.mean(nImat)

        ''' calculate directions '''
        noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
        dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
        for i in range(nrank):
            # E 0 M 1 N
            noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
            noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
        # m1n1
        covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
        _,dirvecE=la.eig(covdirE)
        thetamaxseries[idxeta,ktrial,0,0],thetamaxseries[idxeta,ktrial,0,1]  =np.arctan2(dirvecE[1,0],dirvecE[0,0]),np.arctan2(dirvecE[1,1],dirvecE[0,1])
        _,dirvecI=la.eig(covdirI)  
        thetamaxseries[idxeta,ktrial,1,0],thetamaxseries[idxeta,ktrial,1,1]  =np.arctan2(dirvecI[1,0],dirvecI[0,0]),np.arctan2(dirvecI[1,1],dirvecI[0,1])   
            


normalization m & n
1.0000000000000004 5.147815070493496
eigenvalues: [-2.5+0.j]
0.001227239821432663 0.0
0.00014339055867298723 0.0
0.02495010281058184 0.1
0.02477375497195556 0.1
0.0503014678050254 0.2
0.05129435925661784 0.2
0.07552423791026287 0.30000000000000004
0.07508563384702657 0.30000000000000004
0.10061422565256953 0.4
0.10160230932316633 0.4
0.12561591656847354 0.5
0.12624440491629857 0.5
0.15061372470899181 0.6000000000000001
0.1498174873527194 0.6000000000000001
0.17537410487754088 0.7000000000000001
0.1757381529273681 0.7000000000000001
0.2010974541985376 0.8
0.1997716373153934 0.8
0.22493985566065955 0.9
0.22541027152877782 0.9
0.25090311512921964 1.0
0.2506423903734972 1.0


In [55]:
for idxeta, eta in enumerate(symetaseries): 
    ktrial=1

    meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
    meigvecloading,neigvecloading=mvecseries[idxeta,ktrial,:,0].copy(),nvecseries[idxeta,ktrial,:,0].copy()
    meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
    scale_std=3.0
    for irank in range(nrank):
        # m vector
        # puring E
        mEmat = np.squeeze(meigvecloading[:NE,irank])
        meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
        varmE = mEmat - meanmEtotal
        idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
        mEmat[idxwhere]=meanmEtotal
        meigvecloading[:NE,irank]=mEmat
        meanmE[irank]=np.mean(mEmat)

        # puring I
        mImat = np.squeeze(meigvecloading[NE:,irank])
        meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
        varmI = mImat - meanmItotal
        idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
        mImat[idxwhere]=meanmItotal
        meigvecloading[NE:,irank]=mImat
        meanmI[irank]=np.mean(mImat)
        
        # n vector
        # puring E
        nEmat = np.squeeze(neigvecloading[:NE,irank])
        meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
        varnE = nEmat - meannEtotal
        idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
        nEmat[idxwhere]=meannEtotal
        neigvecloading[:NE,irank]=nEmat
        meannE[irank]=np.mean(nEmat)

        # puring I
        nImat = np.squeeze(neigvecloading[NE:,irank])
        meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
        varnI = nImat - meannItotal
        idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
        nImat[idxwhere]=meannItotal
        neigvecloading[NE:,irank]=nImat
        meannI[irank]=np.mean(nImat)

    ''' calculate directions '''
    noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
    dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
    for i in range(nrank):
        # E 0 M 1 N
        noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
        noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
    # m1n1
    covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
    _,dirvecE=la.eig(covdirE)
    _,dirvecI=la.eig(covdirI)  
    
    fig,ax=plt.subplots(figsize=(12,3))  
    ax.scatter(neigvecloading[NE:,0],meigvecloading[NE:,0],s=5.0,c='blue',alpha=0.5)
    ax.scatter(neigvecloading[:NE,0],meigvecloading[:NE,0],s=5.0,c='red',alpha=0.5)

    ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='r',linestyle='--',linewidth=1.5)
    ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='r',linestyle='--',linewidth=1.5)
    ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color='b',linestyle='--',linewidth=1.5)
    ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color='b',linestyle='--',linewidth=1.5)

    mE,nE=meigvecloading[:NE,0].copy(),neigvecloading[:NE,0].copy()
    confidence_ellipse(np.real(nE),np.real(mE),ax,edgecolor='orange')
    ax.scatter(nE,mE,s=10,c='red',alpha=0.5)
    mI,nI=meigvecloading[NE:,0].copy(),neigvecloading[NE:,0].copy()
    confidence_ellipse(np.real(nI),np.real(mI),ax,edgecolor='cyan')
    ax.scatter(nI,mI,s=10,c='blue',alpha=0.5)
    ax.axvline(c='grey', lw=1)
    ax.axhline(c='grey', lw=1)
    xxx=np.linspace(0.0,1.0,10)
    yyy=np.tan(thetamaxseries[idxeta,ktrial,0,0])*xxx
    ax.plot(xxx,yyy,color='green',linestyle='--',linewidth=2.0)
    # ax.set_xlim(xlims)
    # ax.set_ylim(ylims)
    # ax.set_xticks(xticks)
    # ax.set_yticks(yticks)
    ax.set_aspect('equal')
            


(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)


In [54]:
print(i)

0
